In [4]:
from collections import Counter
import operator
import math
import pandas as pd
import re
import numpy as np
from pandas import Series



def tokenize(doc):
    str1  = re.split('[^А-Яа-я0-9]',doc)
    str1 = [x for x in str1 if x]
    return str1

def tf(document, wordss):
    words = tokenize(document)
    total_words = len(words)
    doc_counter = Counter(words)
    for word in doc_counter:
        doc_counter[word] /= total_words
    tfs = [0 for _ in range(len(wordss))]
    for term, index in wordss.items():
        tfs[index] = doc_counter[term]
    return tfs


def _count_docs_with_word(word, docs):
    counter = 1
    for doc in docs:
        if word in doc:
            counter += 1
    return counter


def idf(documents, words):
    idfs = [0 for _ in range(len(words))]
    total_docs = len(documents)
    for word, index in words.items():
        docs_with_word = _count_docs_with_word(word, documents)
        idf = 1 + math.log10(total_docs / docs_with_word)
        idfs[index] = idf
    return idfs


def _merge_td_idf(tf, idf, terms):
    return [tf[i] * idf[i] for i in range(len(terms))]


def build_tfidf(withoutnan, document, words):
    doc_tf = tf(document, words)
    doc_idf = idf(withoutnan, words)
    return _merge_td_idf(doc_tf, doc_idf, words)


def cosine_similarity(vec1, vec2):
    def dot_product2(v1, v2):
        return sum(map(operator.mul, v1, v2))

    def vector_cos5(v1, v2):
        prod = dot_product2(v1, v2)
        len1 = math.sqrt(dot_product2(v1, v1))
        len2 = math.sqrt(dot_product2(v2, v2))
        return prod / (len1 * len2)

    return vector_cos5(vec1, vec2)




tf_idf_total = []
data=pd.read_excel("CL_Analytics_test_1.xlsx")
data.dropna(subset=['ОВ1.Выделите три основных риска'], inplace=True)
data.index=range(data.shape[0])

aa=[]
for i in range(data.shape[0]-4):
    str0=data['Test.Response ID'][i]
    str1=data['ОВ1.Выделите три основных риска'][i]
    aa.append({str0:str1})
withoutnan123=[]
for i in range(len(aa)):
    for k in aa[i]:
        kk=k
        if ('1.'or'1)') and ('2.'or'2)')and('3.'or'3)') in aa[i][kk]:
            #if 'риск' in aa[i][kk]:
                withoutnan123.append(aa[i])
keys=[]
for i in range(len(withoutnan123)):
    for qk in withoutnan123[i]:
        keys.append(qk)
val=[]
for i in range(len(withoutnan123)):
    for qk in withoutnan123[i]:
        val.append(withoutnan123[i][qk])
data1 = pd.DataFrame({'Test.Response ID': keys,'ОВ1.Выделите три основных риска': val})

aaa=[]
for i in range(data1.shape[0]):
    str1=data1['ОВ1.Выделите три основных риска'][i]
    aaa.append(str1)
allwords=' '.join(aaa)
str1  = re.split('[^А-Яа-я0-9]',allwords)
str1 = [x for x in str1 if x]
words={}
numb=0
for token in str1:
    if token not in words:
        words[token] = numb
        numb += 1
for document in aaa:
    tf_idf_total.append(build_tfidf(aaa, document, words))
queries = input()
query_tfidf = build_tfidf(aaa, queries, words)
cos=[]
for index, document in enumerate(tf_idf_total):
    cos.append(cosine_similarity(query_tfidf, document))
data2 = pd.DataFrame({'Оценка': cos})
writer = pd.ExcelWriter('Result.xlsx', engine='xlsxwriter')
data1.join(data2).to_excel(writer, sheet_name='ABC')
workbook  = writer.book
worksheet = writer.sheets['ABC']
writer.save()